In [1]:
! pip install pymongo

In [2]:
from mongo import MongoDBManager
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import re

# Retrieving data

In [3]:
load_dotenv()
uri = os.getenv('MONGODB_URI')
database_name = os.getenv('DATABASE_NAME')
mongo_manager = MongoDBManager(uri, database_name)

In [4]:
documents = mongo_manager.read_all_documents_in_collection_with_GridFS()

In [5]:
def data_recurcive_length(data):
    total = 0
    try:
        for key in data.keys():
            for profession in data[key].keys():
                for language in data[key][profession].keys():
                    print(f"{key} - {profession} - {language} : {len(data[key][profession][language])}")
                    total += len(data[key][profession][language])
        print(f"Total: {total}")
    except Exception as e:
        print(e)
        print(data[key][profession])

In [6]:
data_recurcive_length(documents)

US - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 68
US - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 18
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 27
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 2723
US - Actuaire - en : 108
US - Adjointe ou adjoint à la recherche - en : 236
US - Administratrice ou administrateur - fr : 8
US - Administratrice ou administrateur - en : 1678
US - Administratrice ou administrateur de contrats - fr : 25
US - Administratrice ou administrateur de contrats - en : 522
US - Administratrice ou administrateur de programmes de développement durable - fr : 12
US - Administratrice ou administrateur de programmes de développement durable - en : 1024
US - Administratrice ou administrateur de prêts commerciaux - fr : 51
US - Administratrice ou administrateur de prêts commerciaux - en : 999
US - Agent ou agente d’approvisionnement et de 

# Formating Datas

In [7]:
def data_to_sql_format(data):
    data_formated = []
    for country in documents.keys():
        for profession in documents[country].keys():
            for language in documents[country][profession].keys():
                    for data in documents[country][profession][language]:
                        data['country'] = country
                        data['profession'] = profession
                        data['language'] = language
                        data_formated.append(data)
    return data_formated                    
data_formated=data_to_sql_format(documents)
print(len(data_formated))

37141


In [10]:
import dedupe

def clear_data(data):
    df=pd.DataFrame(data)
    df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)
    df=df.drop_duplicates()
    df.describe()
    countries = df["country"].drop_duplicates().to_list()
    professions = df["profession"].drop_duplicates().to_list()
    languages = df["language"].drop_duplicates().to_list()
    clear_data={}
    for country in countries:
        for profession in professions:
            for lang in languages:
                try:
                    if country not in clear_data:
                        clear_data[country] = {}
                    if profession not in clear_data[country]:
                        clear_data[country][profession] = {}
                    
                    filtered_df = df[(df["language"] == lang) & (df["profession"] == profession) & (df["country"] == country)]
                    clear_data[country][profession][lang] = filtered_df.to_dict('records')
                except Exception as e:
                    print(e)
                
    return clear_data

data_cleared = clear_data(data_formated)

C:\Users\PC\AppData\Local\Temp\ipykernel_26964\1776475680.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)


In [11]:
data_recurcive_length(data_cleared)

US - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 68
US - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 10
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 2723
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 27
US - Actuaire - en : 84
US - Actuaire - fr : 0
US - Adjointe ou adjoint à la recherche - en : 236
US - Adjointe ou adjoint à la recherche - fr : 0
US - Administratrice ou administrateur - en : 1678
US - Administratrice ou administrateur - fr : 8
US - Administratrice ou administrateur de contrats - en : 409
US - Administratrice ou administrateur de contrats - fr : 21
US - Administratrice ou administrateur de programmes de développement durable - en : 1024
US - Administratrice ou administrateur de programmes de développement durable - fr : 12
US - Administratrice ou administrateur de prêts commerciaux - en : 999
US - Administratrice ou administrateur de prêt

# Visualizing Data

In [12]:
df = pd.DataFrame(data_formated)
df.describe()

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,37141,37141,37141,37141,37141,36119,37141,37141,37141,37141,37141,37141,37141,37141
unique,14897,7710,6530,5413,11539,61,3233,322,584,3164,15,3,17,2
top,Office Administrator,,"Montréal, QC",Full-time,,,,[],,[],2024-09-13,Canada,"Agente commerciale ou agent commercial, politi...",en
freq,450,291,924,4084,296,4487,17621,8222,26759,17670,9015,15684,6458,27069


In [6]:
df[df['shiftAndSchedule'] != ''].describe()

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,9205,9205,9205,9205,9205,8959,9205,9205,9205,9205,9205,9205,9205,9205
unique,4842,2463,2329,1953,2935,52,1177,120,545,1429,10,2,14,2
top,Office Administrator,PHSA,"Montréal, QC",Full-time,POSTE AVEC BEAUCOUP DE DÉPLACEMENT\n\nSERVICES...,,,,[Monday to Friday],[],2024-09-13,Canada,Administratrice ou administrateur,en
freq,220,187,227,769,44,960,3237,1644,1191,2940,3213,5360,1725,7664


In [7]:
df[:5]

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
0,"Agent ou agente de planification, de programma...",CISSSCA,"Lévis, QC","De 29,50 $ à 52,28 $ de l’heure - Temporaire, ...",Présentation de l'organisation\nLe Centre inté...,2024-08-30,"De 29,50 $ à 52,28 $ de l’heure","[Temporaire, Temps plein]",,"[Jours fériés payés, Programme d'Aide aux Empl...",2024-09-09,Canada,"Agente ou agent communautaire, de développemen...",fr
1,"Agent ou agente de planification, de programma...",CIUSSS de la Capitale-Nationale,"Ville de Québec, QC•Travail hybride","Permanent, Temps plein",Sommaire du rôle et des responsabilités :\n\nS...,2024-09-06,,"[Permanent, Temps plein]",Quart de jour\nDu Lundi au Vendredi,[],2024-09-09,Canada,"Agente ou agent communautaire, de développemen...",fr
2,Agente ou agent de recherche et de planificati...,Université Laval,"2705 ​​Laurier Boulevard, Ville de Québec, QC","Temporaire, Temps plein",Agente ou agent de recherche et de planificati...,2024-09-06,,"[Temporaire, Temps plein]",,"[Heures d’arrivée et de départ flexibles, Jour...",2024-09-09,Canada,"Agente ou agent communautaire, de développemen...",fr
3,Directeur(trice) des ventes,Mont-Joli Chrysler,"1325 Boul Jacques-Cartier, Mont-Joli, QC","Permanent, Temps plein",Notre concessionnaire Mont-Joli Chrysler est p...,2024-09-06,,"[Permanent, Temps plein]",,[],2024-09-09,Canada,"Agente ou agent communautaire, de développemen...",fr
4,"Agent ou agente de planification, de programma...",CISSSCA,"Lévis, QC","De 29,50 $ à 52,28 $ de l’heure - Permanent, T...",Présentation de l'organisation\nLe Centre inté...,2024-08-10,"De 29,50 $ à 52,28 $ de l’heure","[Permanent, Temps plein]",,"[Jours fériés payés, Programme d'Aide aux Empl...",2024-09-09,Canada,"Agente ou agent communautaire, de développemen...",fr


In [8]:
df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)
df = df.drop_duplicates()

df = df[((df['positionTitle'] != '') & (df['description'] != '')) | df['positionTitle'] != '']
"""
df['jobType'] = df.apply(
    lambda x: tuple(list(x['jobType']) + list(x['salary'])) 
    if not re.search(r'\d+', str(x['salary'])) 
    else x['jobType'], 
    axis=1
)
"""
"""
df['salary'] = df['salary'].map(
    lambda x: '' 
    if not re.search(r'\d+', str(x))
    else x
)

df['salary'] = df.apply(
    lambda x: str(x['salary']+"-"+str(list(x['jobType'])[0]) )
    if not re.search(r'\d+', str(list(x['jobType'])[0])) 
    else str(x['salary']), 
    axis=1
)
professions[5:7]

df['jobType'] = df['jobType'].map(
    lambda x: '' 
    if not re.search(r'\d+', str(x))
    else x
)
"""
df.describe()

C:\Users\gueid\AppData\Local\Temp\ipykernel_28028\3406113667.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)


,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,31946,31946,31946,31946,31946,31097,31946,31946,31946,31946,31946,31946,31946,31946
unique,12526,6913,5991,4822,10061,52,2817,299,546,2864,12,3,14,2
top,Office Administrator,PHSA,"Montréal, QC",Full-time,,,,(),,(),2024-09-13,Canada,"Agente commerciale ou agent commercial, politi...",en
freq,445,251,754,3414,213,3914,14728,7241,22831,15215,8711,13504,6428,23949


In [10]:
for column in df.columns:
    print(column)

positionTitle
companyName
location
salaryInfoAndJobType
description
datePosted
salary
jobType
shiftAndSchedule
jobBenefits
updatedDate
country
profession
language


In [9]:
df[df['positionTitle'].str.contains('rn', case=False)==True]['positionTitle'].tolist()

['Registered Nurse/Registered Psychiatric Nurse (RN & RPN), Emergency Department / Mental Health – BC Children’s Hospital',
 'Senior Director, Learning & Development / Directeur(trice) principal(e), Formation et développement',
 'RN / RPN',
 'Registered Nurse/Registered Psychiatric Nurse (RN & RPN), Emergency Department / Mental Health – BC Children’s Hospital',
 'Registered Nurse (RN), Cancer Care - BC Cancer Prince George',
 'Home Care Aide – Ongoing Recruitment (External)',
 'Overnight Youth Worker',
 'Registered Nurse (RN)/ Registered Psychiatric Nurse (RPN), Looking Glass Residence - BC Children’s Hospital',
 'Registered Nurse (RN)/ Registered Psychiatric Nurse (RPN), Looking Glass Residence - BC Children’s Hospital',
 'Personal Support Worker Ongoing Recruitment (External)',
 'Intern and Registered Architects',
 'Registered Nurse (RN) - Endoscopy/Brachytherapy PARR, RT Program - BC Cancer - Vancouver',
 'RN / RPN',
 'Internal Medicine',
 'Internal Medicine',
 'Registered Nurse (R

In [24]:
def plot_bar_chart(df, column):
    plt.figure(figsize=(10, 6))
    for i in range(0 , df[column].nunique() , 20):
        df[column].value_counts()[i:i+20].plot(kind='bar')
        plt.show()

In [37]:
from pandasql import sqldf

In [ ]:
plot_bar_chart(df, 'salaryInfoAndJobType')

In [ ]:
df_counts = df[['jobType','salary']].value_counts()[:20].plot(kind='bar')
# df_counts.plot(kind='scatter', x='positionTitle', y='location', s=df_counts['count'] * 10)
# plt.xticks(rotation=45)
# plt.show()
df_counts

In [ ]:
df['salary'] = df.apply(
    lambda x: "" 
    if not re.search(r'\d+', str(x['salary'])) 
    else x['salary'], 
    axis=1
)

# Afficher les 10 valeurs les plus fréquentes dans salary
df['salary'].value_counts()[:20].plot(kind='bar')

In [ ]:



def transform_position_title(df):
    """
    This function cleans a DataFrame containing job posting information by following these steps:
    Steps:
    1. Remove dates:
    2. Remove salary rates:
    3. Remove hourly rates:
    4. Replace codes with actual position titles where possible:
    5. Rename bad encodings:
    6. Remove job type and shift/schedule information:
    7. Remove location if possible:
    Arguments:
    - df: DataFrame containing the job postings data to be cleaned.
    Returns:
    - A cleaned DataFrame according to the steps above.
    """

    # Remove dates
    date_pattern1 = r'\d{1,2}/\d{1,2}/\d{2,4}'
    date_pattern2 = r'\d{1,2}-\d{1,2}-\d{2,4}'
    date_pattern3 = r'\d{1,2}\.\d{1,2}\.\d{2,4}'
    date_pattern4 = r'\d{1,2}\s\w+\s\d{2,4}'
    date_pattern5 = r'\w+\s\d{1,2},\s\d{4}'
    date_pattern6 = r'\d{4}-\d{2}-\d{2}'
    date_patterns = [date_pattern1, date_pattern2, date_pattern3, date_pattern4, date_pattern5, date_pattern6]
    df = df.map(lambda x: re.sub(pattern, '', x) for pattern in date_patterns)

    # Remove month and year
    month_year_pattern = r'\b(janvier|january|février|february|mars|march|avril|april|mai|may|juin|june|juillet|july|août|august|septembre|september|octobre|october|novembre|november|décembre|december)\s\d{4}\b'
    df = df.map(lambda x: re.sub(month_year_pattern, '', x))

    # Remove salary rates
    pattern_salary_ca = re.compile(r'(?:De\s)?([\d\s]+)\s?\$(?:\s?à\s([\d\s]+)\s?\$\s?)(?:par\s|per\s|annuel\s|yearly\s)?(year|annum|an)\s*', re.IGNORECASE)
    pattern_salary_us = re.compile(r'(?:From\s)?\$\d+(?:,\d{3})*(?:\.\d{2})?(?:–\$\d+(?:,\d{3})*(?:\.\d{2})?)?\s?(?:a|per)\s(hour|day|week|month|year)\s*', re.IGNORECASE)
    pattern_salary_fr = re.compile(r'(?:De\s)?([\d\s]+)\s?€\s?à\s([\d\s]+)\s?€\s?(?:par\s|per\s|annuel\s|yearly\s|annum\s)?(an|year)\s*', re.IGNORECASE)
    salary_patterns = [pattern_salary_ca, pattern_salary_us, pattern_salary_fr]
    df = df.map(lambda x: re.sub(pattern, '', x) for pattern in salary_patterns)

    # Remove hourly rates
    time_units_pattern = r'\b(heure|hrs|hour|day|jour|week|semaine)\b'
    df = df.map(lambda x: re.sub(time_units_pattern, '', x))

    # Replace codes with actual position titles where possible(eg. RN -> Registered Nurse)
    
    # Rename bad encodings

    # Remove job type and shift/schedule information
    jobType_pattern = r'\b(temps\splein|full\s?time|part\s?time|temps\s?partiel|contract|contractuel|permanent|temporaire|temporary|contractual\b'
    shiftAndSchedule_pattern = r'\b(jour|day|soir|evening|nuit|night|weekend|fin\sde\ssemaine|weekend|rotating|rotatif|rotative|rotating\b'
    df = df.map(lambda x: re.sub(jobType_pattern, '', x))
    df = df.map(lambda x: re.sub(shiftAndSchedule_pattern, '', x))

    # Remove location if possible
    


    
    return df

def transform_salary_and_job_type(df):
    """
    This function processes salary and job type information in a DataFrame, ensuring consistency in language and data structure, following these steps:
    Steps:
    1. Extract salary information from combined fields:
    2. Handle missing salary information:
    3. Standardize hybrid data (language and format)
    4. Detect and translate language:
    Arguments:
    - df: DataFrame containing job posting data with salary, job type, and schedule information to be processed.
    Returns:
    - A cleaned DataFrame with standardized salary, job type, schedule, and translated information where necessary.
    """
    pass

def transform_company_name(df):
    """
    This function cleans a DataFrame by identifying and removing bad encodings, specifically sequences of squares or other invalid characters in text fields (e.g., names).

    Steps:
    1. Detect bad encodings:
    2. Remove or replace bad encodings:

    Arguments:
    - df: DataFrame containing text fields where bad encodings (e.g., sequences of squares) need to be removed.

    Returns:
    - A cleaned DataFrame with bad encodings removed from text fields.


    """
    pass
def transform_location(df):
    """
    This function cleans a DataFrame by removing special characters, specifically at the beginning of text fields, and addressing bad encodings.

    Steps:
    1. Remove bad encodings:
    2. Remove special characters at the beginning of text fields:

    Arguments:
    - df: DataFrame containing text fields where bad encodings and special characters at the beginning of strings need to be removed.

    Returns:
    - A cleaned DataFrame with special characters removed from the beginnings of text fields and bad encodings corrected.
    """

In [ ]:
def transfrom():
    x=transform_position_title(x)
    x=transform_salary_and_job_type(x)
    x=transform_salary(x)
    x=transform_company_name(x)
    x=transform_location(x)

In [ ]:
plot_bar_chart(df, 'positionTitle')

#### We can see that salaries are too different we can try to put them together in intervals